In [ ]:
!pip install pyunpack
!pip install patool

In [ ]:
#Importamos las librerías de costumbre
import numpy as np
import pandas as pd
#Instalamos la librería que nos va a permitir bajar de drive
!pip install gdown
import gdown
#Colocamos la URL del archivo
url = 'https://drive.google.com/uc?id=1tY5Nkrr7mEvkXjrlWPHyzx9DoDZhSD2l'
#Colocamos el nombre al archivo donde la información será guardada
Nombre = 'Vehiculos.rar'
#Lo descargamos
gdown.download(url,Nombre,quiet=False)

In [8]:
#Importación de librerías necesarias

import matplotlib.pyplot as plt
import os
import tensorflow as tf
from pyunpack import Archive
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import json

In [9]:
#Descompresión de las imágenes

os.mkdir('dataset')
Archive('Vehiculos.rar').extractall('dataset/')

In [ ]:
#Se cargan los conjustos de imágenes y se reescalan

train_dir = os.path.join('dataset/Vehiculos/')

BATCH_SIZE = 15
IMG_SIZE = (224, 224)

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             label_mode='binary')


In [ ]:
#Se crea el dataset de validación

train_batches = tf.data.experimental.cardinality(train_dataset)
validation_dataset = train_dataset.take(train_batches // 4)
train_dataset = train_dataset.skip(train_batches // 4)

print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of train batches: %d' % tf.data.experimental.cardinality(train_dataset))

In [ ]:
#Configuración de los conjuntos de datos para mejor el rendimiento en el entrenamiento usando la API tf.data

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
#Aumento de datos

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
#Cambiar la escala de los valores de píxeles de las imágenes 

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [ ]:
# Creación sel modelo base a partir del modelo previamente entrenado MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

#Congelar la base convolucional
base_model.trainable = False

In [ ]:
#Agregar un encabezado de clasificación

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1,activation = "sigmoid",kernel_initializer='random_normal',bias_initializer='zeros')

In [ ]:
def write_json(dataz, filename): 
	with open(filename,'w') as f: 
	    json.dump(dataz, f, indent=4) 

data = []

with open('history.json', 'w') as file:
    json.dump(data, file, indent=4)

In [ ]:
#Compilar el modelo y entrenarlo

os.mkdir('modelos')
base_learning_rate = 0.005
history = []
contador = 0
for i in range(1,101):
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    mcp_save = ModelCheckpoint('/content/modelos/transfer_learning_V_'+str(i)+'.hdf5', save_best_only=True, monitor='val_loss', mode='min',save_weights_only=False)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    history.append(model.fit(train_dataset,epochs=100,callbacks=[mcp_save],validation_data=validation_dataset))

    with open('history.json') as json_file: 
	    datay = json.load(json_file) 
     
	    temp = datay

	    y = { 'accuracy': history[contador].history['accuracy'],
            'loss': history[contador].history['val_loss']
      }        		     

	    temp.append(y) 
    write_json(datay, 'history.json')  
 
    contador = contador + 1
    del model

Epoch 1/100
342/342 [==============================] - 21s 34ms/step - loss: 0.1648 - accuracy: 0.9373 - val_loss: 0.1723 - val_accuracy: 0.9386
Epoch 2/100
342/342 [==============================] - 10s 30ms/step - loss: 0.0666 - accuracy: 0.9738 - val_loss: 0.1458 - val_accuracy: 0.9468
Epoch 3/100
342/342 [==============================] - 11s 32ms/step - loss: 0.0588 - accuracy: 0.9797 - val_loss: 0.1236 - val_accuracy: 0.9570
Epoch 4/100
342/342 [==============================] - 11s 31ms/step - loss: 0.0734 - accuracy: 0.9728 - val_loss: 0.1486 - val_accuracy: 0.9529
Epoch 5/100
342/342 [==============================] - 11s 31ms/step - loss: 0.0570 - accuracy: 0.9797 - val_loss: 0.1074 - val_accuracy: 0.9625
Epoch 6/100
342/342 [==============================] - 11s 31ms/step - loss: 0.0615 - accuracy: 0.9806 - val_loss: 0.1160 - val_accuracy: 0.9652
Epoch 7/100
342/342 [==============================] - 11s 31ms/step - loss: 0.0531 - accuracy: 0.9821 - val_loss: 0.1450 - val_ac

In [ ]:
#Visualización de los resultados en gráficas

for i in range(len(history)):
  plt.plot(history[i].history['accuracy'])
plt.title('Exactitud del modelo')
plt.ylabel('Exactitud')
plt.xlabel('Época')
plt.show()
# "Loss"
for i in range(len(history)):
  plt.plot(history[i].history['val_loss'])
plt.title('Métrica de pérdida')
plt.ylabel('Pérdida')
plt.xlabel('Época')
plt.show()

In [ ]:
#calculo del porcentajes de validación 

best_val_history = []
for i in range(1,101):
    model = load_model('modelos/transfer_learning_'+str(i)+'.hdf5')
    best_val_history.append(model.evaluate(validation_dataset))
    del model

In [ ]:
#Calculo de promedios de validación y pérdida 

prom_loss = 0.0;
prom_acc = 0.0;
for i in range(len(best_val_history)):
    prom_loss = prom_loss + best_val_history[i][0]
    prom_acc = prom_acc + best_val_history[i][1]

prom_loss = prom_loss / len(best_val_history)
prom_acc = prom_acc / len(best_val_history)

print(prom_loss)
print(prom_acc)

In [ ]:
import shutil
shutil.make_archive("modelos_vehiculo", 'zip', "modelos")